# Stereo

Depth Map from Stereo Images

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

# Load left and right images
SAMPLES_DATA_DIR = 'C:/opencv/sources/samples/data/'
# Load and downscale images for faster processing (M×N  image becomes M/2×N/2 image)
imgL = cv2.pyrDown( cv2.imread(SAMPLES_DATA_DIR+'aloeL.jpg') )
imgR = cv2.pyrDown( cv2.imread(SAMPLES_DATA_DIR+'aloeR.jpg') )

    
# Create an StereoMatcher object using semi-global block matching algorithm
# minDisparity: Minimum possible disparity value
# numDisparities: Maximum disparity minus minimum disparity
# blockSize: Matched block size
# P1: The first parameter controlling the disparity smoothness
# P2: The second parameter controlling the disparity smoothness
# P1 is the penalty on the disparity change by plus or minus 1 between neighbor pixels
# P2 is the penalty on the disparity change by more than 1 between neighbor pixels
# disp12MaxDiff: Max allowed difference (in integer pixel units) in the left-right disparity check
# uniquenessRatio: % by which the minimum computed cost function value should win”the second best value 
min_disp = 0
window_size = 3
num_disp = 112-min_disp
p1 = 8*3*window_size**2
p2 = 32*3*window_size**2
stereo = cv2.StereoSGBM_create(minDisparity = min_disp, numDisparities = num_disp,
        blockSize = 16, P1 = p1, P2 = p2,
        disp12MaxDiff = 1, uniquenessRatio = 10)

# Compute the disparity map
disparity = stereo.compute(imgL,imgR)

# Display
plt.subplot(1,2,1),plt.imshow(imgL),plt.xticks([]), plt.yticks([]),plt.title('left image')
plt.subplot(1,2,2),plt.imshow(imgR),plt.xticks([]), plt.yticks([]),plt.title('right image')
plt.show()
plt.imshow(disparity,'hot'),plt.xticks([]), plt.yticks([]),plt.title('disparity')
plt.show()

Epipolar Geometry

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load left and right images
SAMPLES_DATA_DIR = 'C:/opencv/sources/samples/data/'
imgL = cv2.imread(SAMPLES_DATA_DIR+'left08.jpg',cv2.IMREAD_GRAYSCALE)  #queryimage # left image
imgR = cv2.imread(SAMPLES_DATA_DIR+'right08.jpg',cv2.IMREAD_GRAYSCALE) #trainimage # right image

# We need to find as many possible matches between two images to find the fundamental matrix
# We'll use SIFT features
sift = cv2.xfeatures2d.SIFT_create()
# create a Brute-Force Matcher
bfMatcher = cv2.BFMatcher(cv2.NORM_L2)

# find the keypoints and descriptors with SIFT
kpL, desL = sift.detectAndCompute(imgL,None)
kpR, desR = sift.detectAndCompute(imgR,None)

# Match descriptors (find the best two matches for each descriptor)
matches = bfMatcher.knnMatch(desL,desR,k=2)

# ratio test as per Lowe's paper
# We'll store the list of best matches from both the images
good = []
ptsL = []
ptsR = []
for i,(m,n) in enumerate(matches):
    if m.distance < 0.6*n.distance:
        good.append(m)
        ptsR.append(kpR[m.trainIdx].pt)
        ptsL.append(kpL[m.queryIdx].pt)
        
# Find the Fundamental Matrix
ptsL = np.int32(ptsL)
ptsR = np.int32(ptsR)
F, mask = cv2.findFundamentalMat(ptsL,ptsR,cv2.FM_LMEDS)

# mask: every element of which is set to 0 for outliers and to 1 for the other points
# We select only inlier points
ptsL = ptsL[mask.ravel()==1]
ptsR = ptsR[mask.ravel()==1]


# helper function to draw epilines
# Epilines corresponding to the points in first image is drawn on second image
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    # img1 shpae
    r,c = img1.shape
    # convert from grayscale to BGR
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    # for each line
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        # pick a random color
        color = tuple(np.random.randint(0,255,3).tolist())
        # 
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        # draw the line and the points
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

# Find the epilines
# Epilines corresponding to the points in first image is drawn on second image

# Find epilines corresponding to points in right image (second image)
# drawing its lines on left image
linesL = cv2.computeCorrespondEpilines(ptsR.reshape(-1,1,2), 2,F)
linesL = linesL.reshape(-1,3)
img5,img6 = drawlines(imgL,imgR,linesL,ptsL,ptsR)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
linesR = cv2.computeCorrespondEpilines(ptsL.reshape(-1,1,2), 1,F)
linesR = linesR.reshape(-1,3)
img3,img4 = drawlines(imgR,imgL,linesR,ptsR,ptsL)

# Display
plt.subplot(121),plt.imshow(img5),plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(img3),plt.xticks([]), plt.yticks([])
plt.show()

# Optical Flow

Dense Optical Flow: Computing the optical flow for all the points in the frame

In [18]:
import cv2
import numpy as np

# Video reader
SAMPLES_DATA_DIR = 'C:/opencv/sources/samples/data/'
videoreader = cv2.VideoCapture(SAMPLES_DATA_DIR+"vtest.avi")

# Read a single frame (the first frame)
# read() also returns a bool (True/False). If frame is read correctly, it will be True
ret, currentframeRGB = videoreader.read()
currentframeRGB = cv2.pyrDown(currentframeRGB)
# convert to grayscale
currentframe = cv2.cvtColor(currentframeRGB,cv2.COLOR_BGR2GRAY)

# To display the flow, we'll map different flow directions into different colors
# The flow magnitude determines the color intensity
# Much easier to do this using HSV color space
# Hue <- flow direction
# Value <- flow magnitude
# Saturation <- constant (not encoding any information in the saturation channel)
# Initialize the hsv image
hsv = np.zeros_like(currentframeRGB)
# set the saturation channel to 255 for all pixels
hsv[:,:,1] = 255

while(1):
    # Update the previous frame to the current frame
    previousFrame = currentframe.copy()
   
    # Read the next frame
    ret, currentframeRGB = videoreader.read()
    currentframeRGB = cv2.pyrDown(currentframeRGB)
    # convert to grayscale
    currentframe = cv2.cvtColor(currentframeRGB,cv2.COLOR_BGR2GRAY)

    # Compute optical flow using the Gunner Farneback’s algorithm
    # calcOpticalFlowFarneback(prev,next,pyr_scale,levels,winsize,iterations,poly_n,poly_sigma,flags)
    # pyr_scale: parameter specifying the image scale (<1) to build pyramids
    # levels: number of pyramid layers including the initial image
    # winsize: averaging window size
    # iterations: number of iterations the algorithm does at each pyramid level
    # poly_n: size of the pixel neighborhood used to find polynomial expansion in each pixel
    # poly_sigma: standard deviation of the Gaussian that is used to smooth derivatives
    # flags: OPTFLOW_USE_INITIAL_FLOW uses the input flow as an initial flow approximation
    # flow has the same size as prev with two channels: horizontal (0) and vertical (1) flow components
    flow = cv2.calcOpticalFlowFarneback(previousFrame,currentframe, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute the mag/ang flow from the horizontal/vertical flow
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1])
    
    # Set the hue to the angular flow (in radians)
    hsv[:,:,0] = (ang * 180 /np.pi) / 2
    
    # Set the intensity to the normalized mag flow
    hsv[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # Convert to RGB for display
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    # Display
    cv2.imshow('Original',currentframeRGB)
    cv2.imshow('OpticalFlow',rgb)
    # Wait for 100ms 
    k = cv2.waitKey(1000) & 0xff
    # Terminate on pressing the 'escape' key 
    if k == 27:
        break

# Destroy the video reader    
videoreader.release()
# Destroy all windows
cv2.destroyAllWindows()

Lucas-Kanade Optical Flow: Tracking conrner-like points in a video

In [19]:
import numpy as np
import cv2

# Video reader
SAMPLES_DATA_DIR = 'C:/opencv/sources/samples/data/'
videoreader = cv2.VideoCapture(SAMPLES_DATA_DIR+"vtest.avi")

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 15 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Read a single frame (the first frame)
ret, currentframeRGB = videoreader.read()
# convert to grayscale
currentframe = cv2.cvtColor(currentframeRGB,cv2.COLOR_BGR2GRAY)

# Find corners in the first frame
p0 = cv2.goodFeaturesToTrack(currentframe, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(currentframeRGB)

while(1):
    # Update the previous frame to the current frame
    previousFrame = currentframe.copy()
   
    # Read the next frame
    ret, currentframeRGB = videoreader.read()    
    # convert to grayscale
    currentframe = cv2.cvtColor(currentframeRGB,cv2.COLOR_BGR2GRAY)
    

    # calculate optical flow
    # we pass the previous frame, previous points and the current frame
    # p1: current points 
    # st: status numbers which has a value of 1 if next point is found, else zero
    p1, st, err = cv2.calcOpticalFlowPyrLK(previousFrame, currentframe, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        currentframeRGB = cv2.circle(currentframeRGB,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(currentframeRGB,mask)


     # Display
    cv2.imshow('overlaid frame',img)
    # Wait for 100ms 
    k = cv2.waitKey(500) & 0xff
    # Terminate on pressing the 'escape' key 
    if k == 27:
        break

    # Now update the previous points
    p0 = good_new.reshape(-1,1,2)

# Destroy the video reader    
videoreader.release()
# Destroy all windows
cv2.destroyAllWindows()